In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from torchvision import models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import gc
import os 

# clear cuda memory and collect garbage
gc.collect()
torch.cuda.empty_cache()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device: ',device)

x=np.load('scaled_spec_resampled_array.npy')
y=np.load('labels_array.npy')-1
x = x.reshape(x.shape[0], 1, x.shape[1], x.shape[2])

print(x.shape, y.shape)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

class MyDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

train_dataset = MyDataset(x_train, y_train)
test_dataset = MyDataset(x_test, y_test)


train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Device:  cuda:0
(1754, 1, 2048, 80) (1754,)


In [2]:
resnet34 = models.resnet34(pretrained=True)
resnet34.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnet34.fc = nn.Linear(in_features=512, out_features=6, bias=True)    


#check for checkpoint save and load resnet34.pth
if os.path.exists('resnet34.pth'):
    resnet34.load_state_dict(torch.load('resnet34.pth'))
    print('Model loaded')
else:
    print('No model found, loading pretrained model')


resnet34 = resnet34.to(device)

#freeze every layer except the first conv1 and last fc layers of resnet50
'''for name, param in resnet34.named_parameters():
    if name not in ['conv1.weight', 'fc.weight', 'fc.bias']:
        param.requires_grad = False
    else:
        param.requires_grad = True
'''

c:\Users\giaco\miniconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\giaco\miniconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


No model found, loading pretrained model


"for name, param in resnet34.named_parameters():\n    if name not in ['conv1.weight', 'fc.weight', 'fc.bias']:\n        param.requires_grad = False\n    else:\n        param.requires_grad = True\n"

In [3]:
def test(model, test_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_loss = running_loss / len(test_loader)
    val_acc = correct / total
    return val_loss, val_acc

def train(model, train_loader, test_loader, criterion, optimizer, epochs):
    model.train()
    running_loss = 0.0
    epoch_bar = tqdm(range(epochs), position=0)
    for epoch in epoch_bar:
        batch_bar=tqdm(enumerate(train_loader, 0), total=len(train_loader), position=1, leave=False)
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            batch_bar.set_description('Train loss: %.3f' % (loss.item()))
        train_loss = running_loss / len(train_loader)
        epoch_bar.set_description('Train loss: %.3f' % train_loss)
        val_loss, val_acc = test(model, test_loader, criterion)
        print('Epoch: %d, Train Loss: %.3f, Val Loss: %.3f, Val Acc: %.3f' % (epoch, train_loss, val_loss, val_acc))
    return model

In [4]:
optimizer = optim.Adam(resnet34.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

resnet34=train(resnet34, train_loader,test_loader, criterion, optimizer, 100)

torch.save(resnet34.state_dict(), 'resnet34.pth')


Train loss: 1.206:   1%|          | 1/100 [00:24<40:59, 24.84s/it]

Epoch: 0, Train Loss: 1.206, Val Loss: 14.254, Val Acc: 0.279


Train loss: 29.489:   2%|▏         | 2/100 [00:44<35:29, 21.73s/it]

Epoch: 1, Train Loss: 29.489, Val Loss: 1.894, Val Acc: 0.154


Train loss: 31.285:   3%|▎         | 3/100 [01:06<35:33, 21.99s/it]

...

Epoch: 97, Train Loss: 74.499, Val Loss: 0.402, Val Acc: 0.883


Train loss: 74.524:  99%|█████████▉| 99/100 [35:51<00:22, 22.91s/it]

Epoch: 98, Train Loss: 74.524, Val Loss: 0.394, Val Acc: 0.895


Train loss: 74.532: 100%|██████████| 100/100 [36:14<00:00, 21.75s/it]


Epoch: 99, Train Loss: 74.532, Val Loss: 0.623, Val Acc: 0.900
